In [1]:
import numpy as np 
import json 
import os 
import time 



In [2]:
path_folder_no_join = "/home/asilla/duongnh/project/Analys_COCO/Dataset_Inference"

In [3]:
def get_dataset_no_join(path_json):
    '''
    Args:
    Returns:
    '''
    list_file_name = os.listdir(path_folder_no_join)
    file_annotation = open(path_json,'r')
    json_data = json.load(file_annotation)
    l_image = json_data['images']
    l_annotations = json_data['annotations']
    l_categories = json_data['categories']
    dict_file_name = {}
    for item_info_image in l_image:
        file_name = item_info_image["file_name"]
        image_id = item_info_image["id"]
        dict_file_name[file_name] = image_id
    l_image_id_no_join = {}
    for file_name in list_file_name:
        image_id = dict_file_name[file_name]
        l_image_id_no_join[image_id] = 1
    cv_annotation = []
    for annotation in l_annotations:
        image_id = annotation["image_id"]
        if image_id in l_image_id_no_join:
            cv_annotation.append(annotation)
    new_dataset = {}
    new_dataset["images"] = l_image.copy()
    new_dataset["annotations"] = cv_annotation
    new_dataset["categories"] = l_categories
    json_str = json.dumps(new_dataset)
    with open("cv_dataset_no_join.json",'w') as file_json:
        file_json.write(json_str)
        
    

In [9]:
def cal_IOU(bbox1, bbox2):
    '''
    Args: bbox: (xtop, ytop, w, h)
    Returns: IOU
    '''
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    boxA = (int(x1), int(y1), int(x1 + w1), int(y1 + h1))
    boxB = (int(x2), int(y2), int(x2 + w2), int(y2 + w2))
    
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [ ]:
def get_medium_size_bbox_no_join():
    

In [ ]:
def find_best_IOU():
    pass

In [ ]:
def writed_bbox_to_dataset():
    #add flag: result_model
    pass

In [51]:
# loop in result model 
# convert to dict image file name
path_result_model = "/home/asilla/duongnh/datasets/Result_model.json"
file_annotation_result = open(path_result_model,'r')
json_data_result = json.load(file_annotation_result)
dict_rs = {}
for annotation_rs in json_data_result:
    file_name = annotation_rs["image_id"]
    if file_name not in dict_rs:
        dict_rs[file_name] = [annotation_rs]
    else:
         dict_rs[file_name].append(annotation_rs)


In [52]:
def check_size_bbox(bbox):
    config_threshold_small = 1024
    config_threshold_medium_large = 64 * 64
    config_threshold_large = 9216
    xtop, ytop, width, height = bbox
    size_bbox = width * height 
    if size_bbox >= config_threshold_medium_large:
        return True
    else:
        return False

In [48]:
# loop in image gt 
# convert to dict image file name l_bbox no join > medium , id annotation 
path_annotation = "/home/asilla/duongnh/project/Analys_COCO/cv_dataset_no_join.json"
file_annotation = open(path_annotation,'r')
json_data = json.load(file_annotation)
l_image = json_data['images']
l_annotations = json_data['annotations']
l_categories = json_data['categories']


dict_image_name_gt = {}
dict_image_id2image_name = {}
for info_image in l_image:
    image_id = info_image["id"]
    image_name = info_image["file_name"]
    dict_image_id2image_name[image_id] = image_name

for annotation in l_annotations:
    
    image_id = annotation["image_id"]
    num_keypoints = annotation["num_keypoints"]
    keypoint = annotation["keypoints"]
    bbox = annotation["bbox"]
    file_name = dict_image_id2image_name[image_id]
    if num_keypoints == 0 and check_size_bbox(bbox):
        if file_name not in dict_image_name_gt:
            dict_image_name_gt[file_name] = []
            dict_image_name_gt[file_name].append(annotation)
        else:
            dict_image_name_gt[file_name].append(annotation)

    



In [54]:
import cv2

In [57]:
def visualize_keypoint(image, keypoints, color):
    l_keypoint = []
    # print(len(keypoints))
    for i in range(17):
        x = int(keypoints[i * 3])
        y = int(keypoints[i * 3 + 1])
        l_keypoint.append((x,y))
        if x != 0 and y != 0:
            # print(x,y)
            image = cv2.circle(image, (x,y), 5,color, -1)
    # for pair_point in join_keypoint_categories:
    #     start_point_index, end_point_index = pair_point
    #     if l_keypoint[start_point_index - 1][0] != 0 and l_keypoint[start_point_index - 1][1] != 0 and\
    #         l_keypoint[end_point_index - 1][0] != 0 and l_keypoint[end_point_index - 1][1] != 0:
    #         image = cv2.line(image, l_keypoint[start_point_index - 1], l_keypoint[end_point_index - 1], color, 2)
    return image 

In [56]:
#visualize output model 
path_folder = "/home/asilla/duongnh/project/Analys_COCO/Dataset_Inference/Log_folder_image_raw_no_join"
for filename in dict_rs.keys():
    path_image = path_folder + "/" + filename
    image = cv2.imread(path_image)
    l_annotation_rs = dict_rs[filename]
    # print(annotation_rs[0].keys())
    for annotation in l_annotation_rs:
        keypoints = annotation["keypoints"]
        print(len(keypoints))
        image = visualize_keypoint(image, keypoints, (0,0,255))
cv2.write("Log_visualize_result_model/" +  filename, image)

dict_keys(['image_id', 'category_id', 'keypoints', 'score', 'box', 'idx'])


In [50]:
# check filename == filename:
# find top iou -> cv label add join add flag model 

#rewrite annotation

# file_annotation = open(path_json,'r')
# json_data = json.load(file_annotation)
# l_image = json_data['images']
# l_annotations = json_data['annotations']
# l_categories = json_data['categories']

# dict_file_name = {}
# for item_info_image in l_image:
#     file_name = item_info_image["file_name"]
#     image_id = item_info_image["id"]
#     dict_file_name[file_name] = image_id

# for annotation in l_annotations:
#     image_id = annotation["image_id"] 
#     if image_id not in dict_image_id:
#         dict_image_id[image_id] = []
#         dict_image_id[image_id].append(annotation)
#     else:
#         dict_image_id[image_id].append(annotation)

#Process


write_able_data = json_data.copy()
l_update_annotation = []
count_image_write = 0
for key_gt in dict_image_name_gt.keys():
    
    if key_gt in dict_rs:
        # file_name = key_gt 
        # image_id = dict_file_name[file_name]
        # l_annotation = 
        # print("Done")
        l_annotation_gt = dict_image_name_gt[key_gt]
        l_annotation_rs = dict_rs[key_gt]
        # print(len(l_annotation_gt))
        for index_gt, annotation_gt in enumerate(l_annotation_gt):
            num_kp = annotation_gt["num_keypoints"]
            # print(num_kp)
            best_iou = 0
            index_best_IOU = -1
            for index, annotation_rs in enumerate(l_annotation_rs):
                bbox_rs = annotation_rs['box']
                bbox_gt = annotation_gt['bbox']
                tmp_IOU = cal_IOU(bbox_rs, bbox_gt)
                if tmp_IOU > best_iou:
                    best_iou = tmp_IOU
                    # print(tmp_IOU)
                    index_best_IOU = index
            if index_best_IOU != -1:
                if best_iou > 0.1:
                    count_image_write += 1
                    keypoint = l_annotation_rs[index_best_IOU]["keypoints"]
                    annotation_id = annotation_gt["id"]
                    flag = "result_model"
                    for index, gt_annotation in enumerate(l_annotations):
                        annotation_id_gt = gt_annotation["id"]
                        if annotation_id == annotation_id_gt:
                            print("Found: ", key_gt)
                            l_annotations[index]["keypoints"] = keypoint
                            l_annotations[index]["flag"] = "result_model"
                            l_annotations[index]["num_keypoints"] = 500
                    
# path_annotation = "/home/asilla/duongnh/project/Analys_COCO/cv_dataset_no_join.json"
# file_annotation = open(path_annotation,'r')
# json_data = json.load(file_annotation)
# l_image = json_data['images']
# l_annotations = json_data['annotations']
# l_categories = json_data['categories']      
# write_able_data = {}
write_able_data["annotations"] = l_annotations
json_writeable_str = json.dumps(write_able_data)
with open("Test_v1_merge_data.json",'w') as file_merge:
    file_merge.write(json_writeable_str)
# write            

Found:  000000546388.jpg
Found:  000000400853.jpg
Found:  000000210060.jpg
Found:  000000364243.jpg
Found:  000000534516.jpg
Found:  000000190951.jpg
Found:  000000205570.jpg
Found:  000000546126.jpg
Found:  000000404718.jpg
Found:  000000278582.jpg
Found:  000000523217.jpg
Found:  000000041478.jpg
Found:  000000221543.jpg
Found:  000000412793.jpg
Found:  000000129690.jpg
Found:  000000538054.jpg
Found:  000000554892.jpg
Found:  000000185904.jpg
Found:  000000185904.jpg
Found:  000000189376.jpg
Found:  000000189773.jpg
Found:  000000425874.jpg
Found:  000000075372.jpg
Found:  000000018158.jpg
Found:  000000018158.jpg
Found:  000000124002.jpg
Found:  000000388014.jpg
Found:  000000264044.jpg
Found:  000000423395.jpg
Found:  000000424333.jpg
Found:  000000177149.jpg
Found:  000000558253.jpg
Found:  000000243882.jpg
Found:  000000177953.jpg
Found:  000000243418.jpg
Found:  000000355183.jpg
Found:  000000258085.jpg
Found:  000000258085.jpg
Found:  000000057433.jpg
Found:  000000250113.jpg


In [37]:
print(count_image_write)

2173


In [ ]:
loop in result model 
convert to dict image file name

loop in image gt 
convert to dict image file name l_bbox no join > medium , id annotation 

check filename == filename:
find top iou -> cv label add join add flag model 


